<a href="https://colab.research.google.com/github/gibiee/study_AI/blob/master/Machine%20Learning/3_Softmax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np

x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

x_data = np.asarray(x_data, dtype=np.float32)
y_data = np.asarray(y_data, dtype=np.float32)

In [0]:
nb_classes = 3

W = tf.Variable(tf.random.normal((4, nb_classes)), name='weight')
b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
variables = [W, b]

In [0]:
def logit_fn(X) :
    return tf.matmul(X, W) + b
    
def hypothesis(X): # = softmax
    return tf.nn.softmax(logit_fn(X))

def cost_fn(X, Y):
    # cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.math.log(hypothesis(X)), axis=1)) # 아래와 같은 작업을 수행.
    cost = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_true=Y, y_pred=logit_fn(X), from_logits=True))
    
    return cost

def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        grads = tape.gradient(loss, variables)

    return grads

In [4]:
optimizer =  tf.keras.optimizers.SGD(learning_rate=0.1)

epochs = 1000
for i in range(epochs):
    grads = grad_fn(x_data, y_data)
    optimizer.apply_gradients(zip(grads, variables))
    if i == 0 or (i+1) % 100 == 0:
        print('Loss at epoch %d: %f' %(i+1, cost_fn(x_data, y_data).numpy()))

Loss at epoch 1: 5.432341
Loss at epoch 100: 0.739733
Loss at epoch 200: 0.653553
Loss at epoch 300: 0.594769
Loss at epoch 400: 0.546450
Loss at epoch 500: 0.503857
Loss at epoch 600: 0.464986
Loss at epoch 700: 0.428627
Loss at epoch 800: 0.393806
Loss at epoch 900: 0.359593
Loss at epoch 1000: 0.325081


In [5]:
# x_data를 입력으로 하여 y를 예측
predict_y = hypothesis(x_data)
print(predict_y)
print(tf.argmax(predict_y, 1))  # 예측값
print(tf.argmax(y_data, 1)) # 실제 정답값

tf.Tensor(
[[9.3779199e-05 8.0203656e-03 9.9188590e-01]
 [5.5953494e-04 1.6451599e-01 8.3492446e-01]
 [7.1017885e-06 3.4666851e-01 6.5332437e-01]
 [5.1241118e-06 8.2915682e-01 1.7083804e-01]
 [2.4268119e-01 7.3723841e-01 2.0080443e-02]
 [1.1184762e-01 8.8812292e-01 2.9423183e-05]
 [4.1117954e-01 5.8860505e-01 2.1537689e-04]
 [6.1456454e-01 3.8542199e-01 1.3447621e-05]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 1 1 1 1 0], shape=(8,), dtype=int64)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)


# **클래스로 구현하여 간단하게 동작시키기**

In [14]:
class softmax_classifer(tf.keras.Model):
    def __init__(self, nb_classes):
        super(softmax_classifer, self).__init__()
        self.W = tf.Variable(tf.random.normal((4, nb_classes)), name='weight')
        self.b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
    
    def softmax_regression(self, X):
        return tf.nn.softmax(tf.matmul(X, self.W) + self.b)
    
    def cost_fn(self, X, Y):
        logits = self.softmax_regression(X)
        cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.math.log(logits), axis=1))
        return cost
    
    def grad_fn(self, X, Y):
        with tf.GradientTape() as tape:
            cost = self.cost_fn(x_data, y_data)
            grads = tape.gradient(cost, self.variables)            
            return grads
    
    def fit(self, X, Y, epochs=2000, verbose=500):
        optimizer =  tf.keras.optimizers.SGD(learning_rate=0.1)

        for i in range(epochs):
            grads = self.grad_fn(X, Y)
            optimizer.apply_gradients(zip(grads, self.variables))
            if (i==0) | ((i+1)%verbose==0):
                print('Loss at epoch %d: %f' %(i+1, self.cost_fn(X, Y).numpy()))
            
model = softmax_classifer(nb_classes)
model.fit(x_data, y_data)

Loss at epoch 1: 1.476454
Loss at epoch 500: 0.512035
Loss at epoch 1000: 0.293589
Loss at epoch 1500: 0.206322
Loss at epoch 2000: 0.169160
